In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import ipywidgets as widgets
from IPython.display import display

# 從我們自己的 utils 模組中匯入函式
from utils.data_loader import load_dicom_series
from utils.image_processing import register_images

# Matplotlib 設定，讓影像在 Notebook 中直接顯示
%matplotlib inline


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/anaconda3/envs/myenv/lib/python3.11/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/anaconda3/envs/myenv/lib/python3.11/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

載入病人

In [ ]:
# --- 設定區 ---
# 您只需要修改這一行，可以輸入數字或字串
PATIENT_ID_INPUT = 1  # 您可以換成任何想測試的病人編號，例如 1, 15, "22"

# --- 自動格式化路徑 ---
# 將輸入的數字格式化為三位數的字串 (例如 1 -> "001", 22 -> "022")
PATIENT_ID_TO_TEST = f"{int(PATIENT_ID_INPUT):03d}"

# 資料路徑設定
BASE_DATA_PATH = "data/All_Patients"
T1C_FOLDER_NAME = "Ax T1 FS BH+C"
PDFF_FOLDER_NAME = "FatFrac  3D Ax IDEAL IQ BH"

# --- 載入資料 ---
print(f"正在準備載入病人 ID: '{PATIENT_ID_TO_TEST}' 的資料...")
t1c_dir = os.path.join(BASE_DATA_PATH, PATIENT_ID_TO_TEST, T1C_FOLDER_NAME)
pdff_dir = os.path.join(BASE_DATA_PATH, PATIENT_ID_TO_TEST, PDFF_FOLDER_NAME)

volume_t1c_sitk, array_t1c, _ = load_dicom_series(t1c_dir, f"T1+C ({PATIENT_ID_TO_TEST})")
volume_pdff_sitk, array_pdff, _ = load_dicom_series(pdff_dir, f"PDFF ({PATIENT_ID_TO_TEST})")

# 檢查是否成功載入
if array_t1c is not None and array_pdff is not None:
    print(f"\n單一病人 '{PATIENT_ID_TO_TEST}' 資料載入成功！")
else:
    print(f"\n資料載入失敗，請檢查路徑或病人 ID '{PATIENT_ID_TO_TEST}' 是否存在。")

正在準備載入病人 ID: '001' 的資料...
開始載入 T1+C (001) 從: data/All_Patients/001/Ax T1 FS BH+C
  T1+C (001) 載入成功。 NumPy 陣列形狀: (31, 512, 512)
開始載入 PDFF (001) 從: data/All_Patients/001/FatFrac  3D Ax IDEAL IQ BH
  PDFF (001) 載入成功。 NumPy 陣列形狀: (56, 256, 256)

單一病人 '001' 資料載入成功！


In [ ]:
# --- 執行對位 ---
# 這會呼叫您在 utils/image_processing.py 中定義的函式
array_t1c_resampled = register_images(volume_pdff_sitk, volume_t1c_sitk)

# --- 互動式視覺化對位結果 ---
if array_t1c_resampled is not None:
    
    def view_registered_slices(slice_idx):
        """
        一個根據傳入的 slice_idx 來繪製三張對比圖的函式。
        """
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))

        # 影像 1: 原始 T1 (移動影像)
        # 由於原始 T1 的切片數可能不同，我們需要做邊界檢查
        if slice_idx < array_t1c.shape[0]:
            axes[0].imshow(array_t1c[slice_idx, :, :], cmap='gray')
            axes[0].set_title(f'Original T1c (Moving)\nSlice {slice_idx}')
        else:
            # 如果索引超出原始 T1 的範圍，就顯示一張黑色影像作為提示
            axes[0].imshow(np.zeros_like(array_pdff[0, :, :]), cmap='gray')
            axes[0].set_title(f'Original T1c (Out of Bounds)\nSlice {slice_idx}')
        axes[0].axis('off')

        # 影像 2: PDFF (目標影像)
        axes[1].imshow(array_pdff[slice_idx, :, :], cmap='gray')
        axes[1].set_title(f'PDFF (Fixed Target)\nSlice {slice_idx}')
        axes[1].axis('off')

        # 影像 3: 對位後的 T1
        axes[2].imshow(array_t1c_resampled[slice_idx, :, :], cmap='gray')
        axes[2].set_title(f'Registered T1c\nSlice {slice_idx}')
        axes[2].axis('off')

        plt.suptitle(f'Interactive Registration Viewer (Patient {PATIENT_ID_TO_TEST})', fontsize=16)
        plt.tight_layout()
        plt.show()

    # 建立互動式滑桿，並將它與上面的繪圖函式連結
    widgets.interact(
        view_registered_slices,
        slice_idx=widgets.IntSlider(
            min=0,
            max=array_pdff.shape[0] - 1, # 滑桿範圍以對位目標(PDFF)的總切片數為準
            step=1,
            value=array_pdff.shape[0] // 2, # 初始顯示中間的切片
            description='Slice Index:',
            continuous_update=False, # 拖動滑桿放開後才更新，效能較好
            layout=widgets.Layout(width='80%')
        )
    )
else:
    print("對位失敗或尚未執行，無法顯示互動式檢視器。")

  開始進行影像對位...

---- DEBUG: 正在設定度量方法... ----
---- DEBUG: 已成功設定為 MattesMutualInformation ----

  對位完成。最終度量值: -0.3294


interactive(children=(IntSlider(value=28, continuous_update=False, description='Slice Index:', layout=Layout(w…

In [ ]:
# 從 skimage 匯入 Frangi 需要的函式
from skimage.filters import frangi, threshold_otsu
from skimage import exposure
import cv2

def interactive_frangi_viewer(slice_idx, min_sigma, max_sigma, threshold_offset):
    # 取得對位後的 T1 切片和對應的 PDFF 切片
    t1_slice = array_t1c_resampled[slice_idx, :, :]
    pdff_slice = array_pdff[slice_idx, :, :]
    
    # Frangi 濾波
    t1_slice_normalized = exposure.rescale_intensity(t1_slice.astype(np.float64), in_range='image', out_range=(0, 1))
    
    # 確保 min_sigma <= max_sigma
    if min_sigma > max_sigma:
        max_sigma = min_sigma
        
    sigmas = range(min_sigma, max_sigma + 1, 1)
    vesselness = frangi(t1_slice_normalized, sigmas=sigmas, black_ridges=False)
    
    # 閾值處理
    if np.any(vesselness > 0):
        # Otsu's 閾值 + 手動偏移量
        otsu_thresh = threshold_otsu(vesselness[vesselness > 0])
        final_thresh = otsu_thresh + threshold_offset
    else:
        final_thresh = 0.5
        
    vessel_mask = (vesselness > final_thresh)
    
    # 視覺化
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    axes[0].imshow(t1_slice, cmap='gray')
    axes[0].set_title('Registered T1c')
    
    axes[1].imshow(vesselness, cmap='magma')
    axes[1].set_title(f'Frangi Vesselness\nSigmas: {list(sigmas)}')

    # 建立疊加影像
    pdff_uint8 = exposure.rescale_intensity(pdff_slice, out_range=(0, 255)).astype(np.uint8)
    overlay_rgb = cv2.cvtColor(pdff_uint8, cv2.COLOR_GRAY2BGR)
    overlay_rgb[vessel_mask] = [0, 0, 255] # BGR Red
    
    axes[2].imshow(overlay_rgb)
    axes[2].set_title(f'Vessel Overlay on PDFF\nThreshold: {final_thresh:.4f}')

    for ax in axes:
        ax.axis('off')
    plt.show()


# 建立互動式滑桿
widgets.interact(
    interactive_frangi_viewer,
    slice_idx=widgets.IntSlider(min=0, max=array_pdff.shape[0]-1, step=1, value=array_pdff.shape[0]//2, description='Slice:'),
    min_sigma=widgets.IntSlider(min=1, max=10, step=1, value=3, description='Min Sigma:'),
    max_sigma=widgets.IntSlider(min=1, max=10, step=1, value=7, description='Max Sigma:'),
    threshold_offset=widgets.FloatSlider(min=-0.1, max=0.1, step=0.001, value=0, description='Thresh Offset:')
);

interactive(children=(IntSlider(value=28, description='Slice:', max=55), IntSlider(value=3, description='Min S…